**Primeiramente, utilizei seis algoritmos para o treinamento:**

- LogisticRegression(), 
- RandomForestClassifier(), 
- GradientBoostingClassifier(), 
- SVC(), 
- CatBoostClassifier(),
- keras.Sequential()

**Após avaliar o desempenho de cada modelo com hiperparâmetros padrão, escolhi:**
- RandomForestClassifier() 
- CatBoostClassifier() 

**devido ao seu melhor desempenho. Como ambos são baseados em árvores de decisão, também selecionei**
- keras.Sequential()(Redes Neurais Sequenciais) 

**para incluir um modelo com uma abordagem diferente. Esses três modelos serão passados para o algoritmo de otimização de hiperparâmetros.**


**Para otimizar os hiperparâmetros dos modelos Cat Boost e Random Forest, utilizei o algoritmo BayesSearchCV.Essa abordagem usa Otimização Bayesiana passo a passo para explorar os hiperparâmetros mais promissores no espaço do problema. Para a Redes Neurais Sequenciais, utilizei o algoritmo keras_tuner.RandomSearch,ele vem com algoritmos de Otimização Bayesiana, Hiperbanda e Pesquisa Aleatória integrados e também foi projetado para ser fácil para pesquisadores estenderem a fim de experimentar novos algoritmos de pesquisa.**

# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
import keras
import keras_tuner
from keras import layers
from skopt import BayesSearchCV
import warnings
import pickle
import warnings
import joblib
warnings.filterwarnings("ignore")

2024-07-11 16:07:01.165596: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 16:07:01.226628: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 16:07:01.267230: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 16:07:01.783951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 16:07:26.239025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

# Importacao dos dados

In [2]:
dados_tratados = pickle.load(open('dados/dados_tratados.pickle', 'rb'))

In [3]:
x_treino = dados_tratados['x_treino']
y_treino = dados_tratados['y_treino']
x_teste = dados_tratados['x_teste']
y_teste = dados_tratados['y_teste']

print(x_treino.shape, y_treino.shape, x_teste.shape, y_teste.shape) # x_treino.shape

(59606, 20) (59606,) (15891, 20) (15891,)


# Treinamento do modelo

## Selecionando os modelos 

In [5]:
## criar a rede seqeuncial

model_rede = keras.Sequential()
model_rede.add(layers.Dense(units=40,activation='relu',input_shape=(20,)))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=60,activation='relu'))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=40,activation='relu'))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=1,activation='sigmoid'))
model_rede.summary()
opt = keras.optimizers.SGD(learning_rate=0.01)
model_rede.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy', 'Recall'])
#epochs_hist = model.fit(x_treino, y_treino.values, epochs=100,batch_size=500)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 40)             │           840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         2,460 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         2,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,781 (22.58 KB)

 Trainable params: 5,781 (22.58 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
##
modelos = {
    'logistic_regression': LogisticRegression(),
    'random_forest': RandomForestClassifier(),
    'gradient_boost': GradientBoostingClassifier(),
    'svc': SVC(),
    'cat_boost': CatBoostClassifier(),
    'redes_sequencial': model_rede
}

from sklearn.metrics import classification_report,recall_score
lista_recall = {}
for key,model in modelos.items():
    if key == 'redes_sequencial': 
        epochs_hist = model.fit(x_treino, y_treino.values, epochs=100,batch_size=500,verbose=0)  
        print('O modelo treinado é o:', key)
        y_pred = model.predict(x_teste).squeeze()
        y_pred_redes = (y_pred > 0.5).astype(int)
        lista_recall[key] = recall_score(y_teste,y_pred_redes)
        print(classification_report(y_teste.values, y_pred_redes))  
        print('----------------------------------------------------')
        
    else:
        model = model
        if key == 'cat_boost':
            model_treinado = model.fit(x_treino, y_treino,verbose=0)
        else:
            model_treinado = model.fit(x_treino, y_treino)
        print('O modelo treinado e de:', key)
        print(key, model_treinado.score(x_teste, y_teste))
        print(classification_report(y_teste, model.predict(x_teste)))
        lista_recall[key] = recall_score(y_teste,model.predict(x_teste))
        print('----------------------------------------------------')


O modelo treinado e de: logistic_regression
logistic_regression 0.9821282486942294
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     15518
           1       0.72      0.39      0.51       373

    accuracy                           0.98     15891
   macro avg       0.85      0.70      0.75     15891
weighted avg       0.98      0.98      0.98     15891

----------------------------------------------------
O modelo treinado e de: random_forest
random_forest 0.9889874771883456
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     15518
           1       0.90      0.60      0.72       373

    accuracy                           0.99     15891
   macro avg       0.95      0.80      0.86     15891
weighted avg       0.99      0.99      0.99     15891

----------------------------------------------------
O modelo treinado e de: gradient_boost
gradient_boost 0.9879176892580706
             

**Análise dos Resultados**

Os modelos de Boosting parecem ter um desempenho melhor, sendo os três melhores:

Selecionei os seguintes modelos para fazer a tunagem:
- Cat Boost
- Random Forest

Para ter um modelo diferente, o terceiro modelo será a rede sequencial.

In [7]:
pd.DataFrame(list(lista_recall.items()), columns=['Model', 'Recall']).sort_values(by='Recall', ascending=False)

,Model,Recall
4,cat_boost,0.705094
1,random_forest,0.595174
2,gradient_boost,0.579088
3,svc,0.439678
5,redes_sequencial,0.396783
0,logistic_regression,0.394102


# Tunagem dos modelos

## Cat Boost

In [8]:
## Parametros Cast Boost
parametros_cat = {"iterations": [2, 10, 20, 30, 50, 100, 200, 300, 400, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1],
                    "depth": [1, 3, 5, 7, 9],
                }

bayes_cat = BayesSearchCV(
            CatBoostClassifier(verbose=0), parametros_cat, n_iter=100, cv=3, scoring="recall"
        )
bayes_cat.fit(x_treino, y_treino.values)

BayesSearchCV(cv=3,
              estimator=<catboost.core.CatBoostClassifier object at 0x7f8a600a0e90>,
              n_iter=100, scoring='recall',
              search_spaces={'depth': [1, 3, 5, 7, 9],
                             'iterations': [2, 10, 20, 30, 50, 100, 200, 300,
                                            400, 500],
                             'learning_rate': [0.1, 0.15, 0.2, 0.4, 0.6, 0.8,
                                               1]})

In [19]:
print('Melhores parametros:',bayes_cat.best_params_,'Score:',bayes_cat.best_score_) #bayes_cat.best_params_,bayes_cat.best_score_

Melhores parametros: OrderedDict({'depth': 5, 'iterations': 400, 'learning_rate': 1}) Score: 0.6830625806529422


In [ ]:
cat_model = CatBoostClassifier(**bayes_cat.best_params_)
cat_model.fit(x_treino, y_treino.values)

In [26]:
cat_model.save_model("modelos_salvos/cat_boost_model.cbm")

## Random Forest

In [27]:
parametros_random = {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
            'max_depth': [1,2,60,70,80,90,100],
            'max_features': [1, 3, 4,5,6,8,10],
            'criterion':['gini','entropy'],
                 }
bayes_random = BayesSearchCV(
            RandomForestClassifier(), parametros_random, n_iter=100, cv=3, scoring="recall"
        )
bayes_random.fit(x_treino, y_treino.values)

BayesSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
              scoring='recall',
              search_spaces={'criterion': ['gini', 'entropy'],
                             'max_depth': [1, 2, 60, 70, 80, 90, 100],
                             'max_features': [1, 3, 4, 5, 6, 8, 10],
                             'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80,
                                              90, 100]})

In [28]:
rando_model = RandomForestClassifier(**bayes_random.best_params_)
rando_model.fit(x_treino, y_treino.values)

RandomForestClassifier(max_depth=60, max_features=10, n_estimators=90)

In [32]:
joblib.dump(rando_model, 'modelos_salvos/random_forest_model.pkl')

['modelos_salvos/random_forest_model.pkl']

## Redes Sequencial

In [23]:
def build_model(hp,camadas_ocultas=4):
    model = keras.Sequential()
    # Primeira camada com input_shape especificado corretamente
    model.add(keras.layers.Dense(
          hp.Choice('units', [8, 16, 32,50,64,100]),
          activation='relu',input_shape=(x_treino.shape[1],)))
    # Camadas ocultas adicionais
    for _ in range(camadas_ocultas):
        model.add(keras.layers.Dense(
            hp.Choice('units', [8, 16, 32,50, 64,100]),
            activation='relu'))
    # Camada de saída
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    # Definição do otimizador
    opt = keras.optimizers.SGD(learning_rate= hp.Choice("learning_rate", [0.001, 0.01, 0.02 ,0.05,0.1, 0.2, 0.3]))
    # Compilação do modelo
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', 'Recall'])
    return model

tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,)

In [24]:
tuner.search(x_treino, y_treino, epochs=10, validation_data=(x_teste, y_teste),verbose=0)
best_model = tuner.get_best_models()[0]

In [26]:
best_model.save("modelos_salvos/redes_neurais_model.h5")